# Artificial Intelligence- Supervised learning

**It is recommended to [view this notebook in nbviewer](http://nbviewer.ipython.org/github/rhiever/Data-Analysis-and-Machine-Learning-Projects/blob/master/example-data-science-notebook/Example%20Machine%20Learning%20Notebook.ipynb) for the best viewing experience.**

**You can also [execute the code in this notebook on Binder](https://mybinder.org/v2/gh/rhiever/Data-Analysis-and-Machine-Learning-Projects/master?filepath=example-data-science-notebook%2FExample%20Machine%20Learning%20Notebook.ipynb) - no local installation required.**

## Introduction

[[ go back to the top ]](#Table-of-contents)

Machine learning is an application of artificial intelligence (AI) that provides systems the ability to automatically learn and improve from experience without being explicitly programmed.
In this notebook, we will be using Supervised learning to predict that learns the difference between stocks and which ones will be worth buying, based on financial indicators given by the 10-k fillings.

## The problem domain

[[ go back to the top ]](#Table-of-contents)

In this first approach to this data analysis problem, we need to specify what is the problem we want to solve, as well as defining a measure the success of the solution found. To help us accomplish that, we will go through some questions given by the data analysis checklist.

**Did you specify the type of data analytic question (e.g. exploration, association causality) before touching the data?** 

>Our goal in this project is to classify stocks into 2 classes (0, if the their price will decrease in 2019, or 1 otherwise). This classification will be based on the details of each company given by their 10-k fillings during 2018.

**Did you define the metric for success before beginning?**

>Let's do that now. Since we are performing a classification, we can use accuracy (the fraction of correctly classified classes), to define a metric for the quality of our algorithm. After some research, we discovered that, in the long term, an algorithm that predicts stocks trend with less than 65 % of accuracy will underperform buy and hold strategy. We will try to match or beat this accuracy for the values of 2019.

**Did you record the experimental design?**

>No. We are using a public dataset from [Kaggle](https://www.kaggle.com/cnic92/beat-us-stock-market-data), that contains information about over 630 companies from the technology sector of the US stock market.

**Did you consider whether the question could be answered with the available data?**

>The dataset that we will be using only has information about aprominately 640 companies, all of them from the technology sector, which makes it harder to predict stock trends for the entire stock market.